In [ ]:
from IPython.core.display import HTML

with open('style.html', 'r') as file:
    css = file.read()
HTML(css)

# Einleitung

Bevor in die Umsetzung der Aufgabe eingestiegen wird, werden einige Grundlagen des Schachspiels für den ungeschulten Spieler erklärt. Dadurch können die Schritte zur Berechnung eines optimalen Endspiels besser nachvollzogen werden. 

# Schach

Bei Schach handelt es sich um ein Brettspiel, das insgesamt von zwei Spielern gespielt werden kann.
Im folgenden Abschnitt soll eine Einführung in den Spielablauf gegeben werden. Diese wird benötigt, um die im Verlauf dieses Dokuments vorgestellten Konzepte zu verstehen.

## Spielbrett und Spielfiguren
Ein Schachbrett besteht aus insgesamt 64 Feldern, die in einer 8x8 Matrix angeordnet sind. Dabei werden die Spalten mit
den Buchstaben a-h und die Zeilen mit den Zahlen 1-8 beschriftet. Das typische Schachmuster entsteht durch einen regelmäßigen Wechsel
zwischen weißen und schwarzen bzw. dunklen und hellen Feldern. Dementsprechend sind insgesamt 32 weiße und 32 schwarze Felder
auf dem Schachbrett zu finden. Alle Felder können mithilfe der zuvor genannten Beschriftung eindeutig identifiziert
werden. Wenn in diesem Dokument Felder durch ein Kürzel wie beispielsweise "c5" spezifiziert werden, ist das Feld
gemeint, das in der Spalte "*Buchstabe*" und der Zeile "*Zahl*" zu finden ist. Das vorherige Beispiel befindet sich also in der dritten (c) Spalte und der fünften (5) Zeile.

Im Folgenden werden die Spalten des Schachbretts als Linien und die Zeilen als Reihen bezeichnet.

Dieses Schachbrett wird maximal von 32 Figuren besetzt (16 weiße und 16 schwarze Figuren). Eine Seite besteht aus
insgesamt sechs unterschiedlichen Figuren. Bei diesen Figuren handelt es sich um:

- Bauern (8)
- Springer (2)
- Läufer (2)
- Türme (2)
- Dame (1)
- König (1)

Die Zahl in den Klammern steht hierbei für die Anzahl an Figuren pro Spieler. Die Aufstellung zu Beginn des Spiels kann
aus der folgenden Abbildung entnommen werden:

![Grundaufstellung](Abbildungen/Grundaufstellung.png)

### Bewegungsmöglichkeiten der Figuren  
Jede der bereits genannten Figuren hat einen Wert und ein Bewegungsmuster. 
Figuren können im Rahmen dieses Bewegungsmuster bewegt werden, werden aber durch andere Figuren blockiert. Eine Figur kann nur 
in Ausnahmefällen übersprungen werden und blockiert grundsätzlich die Bewegungen aller anderen Figuren. 
Figuren des Gegners (der anderen Farbe) können geschlagen werden, indem eine eigene Figur auf dasselbe Feld gestellt wird. 
Eine geschlagene Figur wird vom Spielfeld entfernt. Zwei Figuren derselben Farbe können nicht auf demselben Feld platziert werden.
Diese lauten wie folgt (Jonathan Carlstedt, Die kleine Schachschule (2015): S.10ff., S.40):
- **König** (unendlich): Der König gehört zu den unbeweglichsten Figuren auf dem Spielfeld. Er kann pro Zug nur ein Feld entlang
einer Reihe, Linie oder Diagonalen bewegt werden. Dadurch besitzt er jedoch die Möglichkeit, in jede Richtung eine gegnerische Figur
zu schlagen. Als weitere Einschränkung muss beim Ziehen mit dem König beachtet werden, dass das angestrebte Feld nicht durch eine
gegnerische Figur abgedeckt wird. Ein Feld gilt als abgedeckt, wenn eine gegnerische Figur es in einem Zug betreten und die darauf 
stehende Figur schlagen kann. Ist dies der Fall, darf der König nicht auf dieses Feld gesetzt werden.
![König-Bewegungsmuster](Abbildungen/König.png)
- **Turm** (5): Der Turm besitzt die Möglichkeit, in einer Reihe oder Linie beliebig viele Felder zu überqueren
(maximal bis zum Ende des Spielfeldes). 
![Turm-Bewegungsmuster](Abbildungen/Turm.png)
- **Läufer** (3): Der Läufer kann wie ein Turm in geraden Linien bewegt werden. Er unterscheidet sich dadurch,
dass er nur diagonal bewegt werden kann. Ein Läufer auf dem Feld "a1" kann folglich nur nach "b2" oder entlang der Diagonale
bewegt werden.
![Läufer-Bewegungsmuster](Abbildungen/Läufer.png)
- **Dame** (9): Die Dame zählt zu den beweglichsten Figuren auf dem Spielfeld. Sie kombiniert die Bewegungsmuster des
Läufers und des Turms. Das bedeutet, dass sie horizontal (entlang der Reihen), vertikal (entlang der Linien) und diagonal bewegt werden kann.
![Dame-Bewegungsmuster](Abbildungen/Dame.png)
- **Bauer** (1): Der Bauer ist die Figur mit der geringsten Beweglichkeit. Dieser kann nur entlang der Linie nach vorne bewegt werden.
Die erste Bewegung jedes Bauern kann ein oder zwei Felder weit sein, folgende Bewegungen sind immer genau ein Feld weit. 
Eine Besonderheit des Bauerns, liegt in der Richtung, in die ein Bauer gegnerische Figuren schlagen darf. 
Dieser darf nur diagonal nach vorne schlagen. Weiter wird der Bauer in eine
beliebige Spielfigur (außer einem Bauern und einem zweiten König) gewandelt, sobald er die Grundlinie des Gegners erreicht hat.
![Bauer-Bewegungsmuster](Abbildungen/Bauer.png)
- **Springer** (3): Der Springer besitzt im Gegensatz zu allen bereits beschriebenen Figuren keine lineare Bewegungsrichtung.
Er kann um zwei Felder nach vorne und ein Feld zur Seite versetzt werden. Dieses Verfahren gilt in jede Richtung, 
sodass der Springer im Optimalfall acht Felder erreichen kann. Der Name des Springers kommt dadurch
zustande, dass er die einzige Figur ist, die andere Figuren überspringen kann. Nur das "Zielfeld" kann durch eine eigene
Figur blockiert werden. 
![Springer-Bewegungsmuster](Abbildungen/Springer.png)

## Spielablauf

In einem Spiel ziehen die Spieler immer abwechselnd eine Figur ihrer Farbe. Den ersten Zug hat dabei immer weiß. 

Beide Spieler verfolgen während der ganzen Partie das Ziel, den gegnerischen Spieler Schach-Matt zu setzen. Ein Spieler ist Schach-Matt, wenn folgende Bedingungen erfüllt sind:
1. Der König wird durch eine gegnerische Figur bedroht.
2. Der König kann dieser Bedrohung nicht ausweichen.

Eine solche Bedrohung liegt vor, wenn der gegnerische Spieler im nächsten Zug den König schlagen kann. Dies kann auf drei unterschiedliche Weisen pariert werden:
1. Der König bewegt sich aus dem "Schach".
2. Der Spieler schlägt die Schach-gebende Figur.
3. Eine Figur stellt sich zwischen die Schach-gebende Figur und den König.

Ein anderer Spielausgang neben dem Schach-Matt liegt in dem Patt. Ein Patt ist dann gegeben, wenn der Spieler, der am Zug ist, keine Figur mehr ziehen kann und der König des Ziehenden nicht im Schach steht.

Da diese Studienarbeit nicht vorsieht, das komplette Schachspiel zu erklären, werden die restlichen Spielregeln nicht näher erläutert. 
Diese werden aber in der [python-chess](https://python-chess.readthedocs.io/en/latest/) Bibliothek, welche für die Abbildung des Schachspiels im Code verwendet wird, umgesetzt und berücksichtigt.

Anhand der Nummerierung der Notebooks können nun die einzelnen Schritte zur Erstellung und Validierung einer Schach-Endspiel-KI nachvollzogen werden.
Die einzelnen Notebooks begleiten chronologisch die Erstellung, Validierung und Verwendung einer Endspiel-KI. Die hierfür notwendigen Notebooks sind:

1. 02_calculation.ipynb
2. 03_play_against_ai.ipynb
3. 04_play_from_history.ipynb
4. 05_stockfish_compare.ipynb
5. 06_validate_sequences.ipynb

Im Util-Ordner werden zusätzlich allgemeine Importe und Funktionen aufgelistet, die für die Ausführung der aufgelisteten Notebooks benötigt werden.

In der PDF-Version dieser Arbeit ist jedes der aufgelisteten Notebooks ein Kapitel.
